In [1]:
# -----********************-----

# Created Time: 2024/12/09

# Last updated: 2024/12/11

# Author: Tara Liu, Yiyi He

### Use Case

# This notebook explores the application of autoregressive models
# 1. 

# -----********************-----

In [13]:
# Import libraries
import os
import warnings
warnings.filterwarnings("ignore")

# Stats
from statsmodels.tsa.api import ARDL
import statsmodels.api as sm
import numpy as np
from statsmodels.tsa.ardl import ardl_select_order
from statsmodels.tsa.api import VAR

# Geo
from shapely.geometry import Point, Polygon
import geopandas as gpd
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.options.display.max_rows = 1000

# Plot
import matplotlib.pyplot as plt
import seaborn as sns

# Processing
from tqdm import tqdm

In [17]:
# Define functions
def find_lag(df, maxlag):

    time_start = pd.Timestamp.now()
    
    sel_res = ardl_select_order(
        df['pct_blackout'],
        exog=df[['t2m', 'wind_speed', 'tp']],
        maxlag=maxlag,
        ic='aic',
        maxorder=maxlag,
        causal=True,
        trend='ct'
        )
    
    time_selected = pd.Timestamp.now()
    print(f'time elapsed for selecting order: {time_selected-time_start}')
    
    return sel_res.model.ardl_order

In [5]:
# Load input dataframe
df_518 = pd.read_csv("/Users/yiyi/Desktop/df_for_model.csv")
df_518.head(3)

,datetime,pct_blackout,wind_forest_cover,t2m,tp,wind_speed,station_id,climate_zone_code
0,2014-11-17 17:00:00,0.0,0.0,295.09723,5.645638e-04,1.310930,495,Aw
1,2014-11-29 17:00:00,0.0,0.0,293.04870,0.000000e+00,2.215274,495,Aw
2,2014-11-30 22:00:00,0.0,0.0,289.83447,4.351137e-07,2.000125,495,Aw


# FIND LAG

In [29]:
# Create a list of unique station ids
station_id_lst = list(set(df_518.station_id.unique()))
# Initiate an empty dictionary
station_id_lag_dic = {}
# Set max lag
maxlag = 5
# Iterate through all stations
for s_id in tqdm(station_id_lst):
    # Subset station data
    station_df = df_518[df_518['station_id'] == s_id]
    # Find optimum lag and store station id with optimum lag in dictionary
    station_id_lag_dic[s_id] = find_lag(station_df, maxlag)

In [21]:
station_id_lag_dic_2 = station_id_lag_dic

In [33]:
station_lags_df = pd.DataFrame.from_dict(station_id_lag_dic_2, orient='index')
station_lags_df.columns = ['lag_0', 'lag_1', 'lag_2', 'lag_3']
sum(station_lags_df.lag_3.isna())

422

In [34]:
station_lags_df

,lag_0,lag_1,lag_2,lag_3
1,0,1.0,5.0,NaN
2,0,1.0,1.0,4.0
3,0,3.0,3.0,NaN
4,0,1.0,1.0,NaN
5,0,4.0,2.0,NaN
6,0,1.0,2.0,NaN
8,5,4.0,3.0,5.0
9,1,NaN,NaN,NaN
10,1,4.0,NaN,NaN
11,0,3.0,1.0,NaN


# VAR